In [3]:
import symforce

symforce.set_symbolic_api("sympy")

import symforce.symbolic as sf
from symforce.notebook_util import display
from symforce import geo
from symforce import values
from symforce.geo.unsupported.pose3_se3 import Pose3_SE3

def n_link_forward_kinematics(joint_angles: values.Values, screw_axes_params: values.Values, M_params: values.Values, epsilon: sf.Scalar):
    """
    Computes the forward kinematics for an n-link robot using the Product of Exponentials (PoE) formula,
    with symbolic parameters for joint angles, screw axes, and the initial transformation.

    Args:
        joint_angles: A Values object containing symbolic joint angles, with keys "q0", "q1", etc.
        screw_axes_params: A Values object containing symbolic parameters for the screw axes.
                         Each key should be of the form "S{i}", where i is the joint index,
                         and the value should be a 6-vector representing the screw axis [omega_x, omega_y, omega_z, v_x, v_y, v_z].
        M_params: A Values object containing symbolic parameters for the initial transformation matrix M.
                  The keys should be "R" for the rotation (as a 3-vector Euler angles) and "t" for the translation (as a 3-vector).
        epsilon: A small number to handle singularities in Rot3.from_tangent.

    Returns:
        The homogeneous transformation matrix representing the end-effector configuration in the space frame.
    """
    T = sf.Pose3.identity()
    n = len(joint_angles)

    for i in range(n):
        # Extract screw axis parameters and joint angle from Values objects
        S = screw_axes_params[f"S{i+1}"]
        q = joint_angles[f"q{i+1}"]

        # Create se3 element (6x1 vector)
        se3_element = sf.Matrix61(S * q)

        # Create a Pose3_SE3 object from the se3 element
        pose3_se3 = Pose3_SE3.from_tangent(se3_element, epsilon=epsilon)

        # Convert Pose3_SE3 to Pose3
        pose3 = sf.Pose3(R=pose3_se3.R, t=pose3_se3.t)

        # Update the transformation
        T = T * pose3

    # Construct the initial transformation matrix from symbolic parameters
    R = geo.Rot3.from_yaw_pitch_roll(*M_params["R"])
    t = geo.V3(M_params["t"])
    M = geo.Pose3(R=R, t=t)

    # Multiply with the initial transformation matrix
    T = T * M

    # Convert to homogeneous matrix at the end
    return T.to_homogenous_matrix()

# Example usage (adjust as needed for your specific case)
joint_angles = values.Values(
    q1=sf.Symbol("q1"),
    q2=sf.Symbol("q2"),
    q3=sf.Symbol("q3"),
)

screw_axes_params = values.Values(
    S1=sf.V6.symbolic("S1"),
    S2=sf.V6.symbolic("S2"),
    S3=sf.V6.symbolic("S3"),
)

M_params = values.Values(
    R=sf.V3.symbolic("R"),
    t=sf.V3.symbolic("t"),
)

epsilon = sf.numeric_epsilon
end_effector_pose = n_link_forward_kinematics(joint_angles, screw_axes_params, M_params, epsilon)

display("End-Effector Pose:\n")
display(f"{end_effector_pose}")

'End-Effector Pose:\n'

'Matrix([\n[                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            

In [6]:
def slider_crank_kinematics(q1: sf.Scalar, q3: sf.Scalar, params: values.Values, epsilon: sf.Scalar):
    """
    Computes the forward kinematics for a slider-crank linkage (1-RRPR) using n_link_forward_kinematics.

    Args:
        q1: Output joint angle (revolute)
        q3: Input prismatic joint displacement
        params: A Values object containing geometric parameters of the mechanism
        epsilon: A small number to handle singularities

    Returns:
        A tuple (T_sb, joint_variables) where T_sb is the end-effector pose and
        joint_variables is a dictionary of all joint variables.
    """
    # Define the kinematic chains using n_link_forward_kinematics
    def chain1(q1, q2):
        joint_angles = values.Values(q1=q1, q2=q2)
        screw_axes = values.Values(
            S1=sf.V6([0, 0, 1, 0, 0, 0]),
            S2=sf.V6([0, 0, 1, -params['l1'], 0, 0])
        )
        M = values.Values(R=sf.V3([0, 0, 0]), t=sf.V3([params['l1'], 0, 0]))
        return n_link_forward_kinematics(joint_angles, screw_axes, M, epsilon)
    
    def chain2(q3, q4):
        joint_angles = values.Values(q3=q3, q4=q4)
        screw_axes = values.Values(
            S1=sf.V6([0, 0, 0, 1, 0, 0]),
            S2=sf.V6([0, 0, 1, 0, 0, 0])
        )
        M = values.Values(R=sf.V3([0, 0, 0]), t=sf.V3([0, 0, 0]))
        return n_link_forward_kinematics(joint_angles, screw_axes, M, epsilon)
    
    # Loop-closure condition
    def loop_closure(q2, q4):
        return chain1(q1, q2) - chain2(q3, q4)

    # Solve for q2 and q4 using loop-closure condition
    q2 = sf.Symbol('q2')
    q4 = sf.Symbol('q4')
    closure_eqs = loop_closure(q2, q4)
    solution = sf.solve(closure_eqs, [q2, q4])

    # Extract solutions
    q2_sol = solution[q2][0]  # Assuming single solution, might need to handle multiple
    q4_sol = solution[q4][0]

    # Compute final end-effector pose
    T_sb = chain1(q1, q2_sol)

    return T_sb, {'q1': q1, 'q2': q2_sol, 'q3': q3, 'q4': q4_sol}

# Example usage
params = values.Values({'l1': sf.Symbol('l1')})
q1 = sf.Symbol('q1')
q3 = sf.Symbol('q3')
epsilon = sf.numeric_epsilon

T_sb, joint_vars = slider_crank_kinematics(q1, q3, params, epsilon)

display("End-Effector Pose:\n")
display(f"{T_sb}")
display("\nJoint Variables:\n")
display(f"{joint_vars}")

KeyError: 'q1'